<a href="https://colab.research.google.com/github/saran-eleven/Deeplearning/blob/main/MCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_csv("malicious_phish_cleaned.csv")
data.head()


,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [5]:
data['type'].value_counts()

,count
type,
benign,428080
defacement,95308
phishing,94086
malware,23645


In [7]:
data['label'] = data['type'].map({
    'benign': 0,
    'phishing': 1,
    'malware': 1,
    'defacement': 1
})


In [9]:
data_final = data[['url', 'label']]
data_final.head()


,url,label
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [11]:
data_final.to_csv("phishing_urls_final.csv", index=False)


STEP 1: Load the CSV and Prepare Data


In [12]:
import pandas as pd

# Load cleaned & labeled dataset
data = pd.read_csv("phishing_urls_final.csv")

# Check first 5 rows
data.head()


,url,label
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


STEP 2: Feature Extraction Using TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Transform URLs into features
X = vectorizer.fit_transform(data['url'])
y = data['label']  # target variable


STEP 3: Split Dataset into Training & Testing

In [18]:
from sklearn.model_selection import train_test_split

# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


STEP 4: Train Machine Learning Model

In [15]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)


LogisticRegression()

🔹 STEP 5: Evaluate the Model

In [16]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test set
y_pred = model.predict(X_test)

# Check accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Detailed report
print(classification_report(y_test, y_pred))

Accuracy: 0.9492684676815573
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     85430
           1       0.97      0.88      0.92     42794

    accuracy                           0.95    128224
   macro avg       0.96      0.93      0.94    128224
weighted avg       0.95      0.95      0.95    128224



Save the Model for Flask Backend

In [17]:
import joblib

# Save model and vectorizer
joblib.dump(model, "phishing_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


['vectorizer.pkl']

In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

model = joblib.load("phishing_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

@app.route("/predict", methods=["POST"])
def predict():
    url = request.json["url"]
    features = vectorizer.transform([url])
    prediction = model.predict(features)[0]

    result = "Phishing" if prediction == 1 else "Safe"
    return jsonify({"result": result})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [ ]:
from google.colab import files

files.download("phishing_model.pkl")
files.download("vectorizer.pkl")


In [ ]:
files.download("app.py")
